# IMDb Data Cleaning Master Class

## 1. Introduction

**The Dataset:**
We are using the official [IMDb Non-Commercial Dataset](https://developer.imdb.com/non-commercial-datasets/). It is a relational dataset split across multiple files.

**The Challenge:**
- **Format:** Files are TSV (Tab Separated Values), not standard CSV.
- **Missing Data:** Missing values are represented by the string `\N`, not empty space.
- **Size:** The full dataset is massive (~9GB uncompressed). We must use efficient loading techniques.

**Our Goal:**
Create a clean, merged dataset of movies to answer questions like: *"What makes a movie successful in the 21st century?"*

---

### ⚙️ Data Setup

The raw data files are not included in the repo (too large). Before running this notebook, you need to download **two files** and place them in the `data/` folder:

- `title.basics.tsv`
- `title.ratings.tsv`

You can download them from https://datasets.imdbws.com/ (they come as `.gz` files, so unzip them after downloading).

Once you have both files in `data/`, you're ready to run the notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# CONFIGURATION
# ------------------------------------------------------
# Show all columns when printing a dataframe (no more ...)
pd.set_option("display.max_columns", None)

# Set a clean plotting style
plt.style.use("ggplot")
%matplotlib inline

## 2. Loading the Data

We will load two key tables:
1. **`title.basics`**: Contains the movie title, release year, runtime, and genre.
2. **`title.ratings`**: Contains the average rating and number of votes.

In [2]:
# Define file paths (Assumes data is in the data/ folder)
basics_path = "../data/title.basics.tsv"
ratings_path = "../data/title.ratings.tsv"

# LOAD DATA
# ------------------------------------------------------
# 1. sep="\t": The file is tab-separated
# 2. na_values="\\N": IMDb uses "\N" for missing data. We tell Pandas to treat this as NaN.

print("Loading Basics Table...")
df_basics = pd.read_csv(basics_path, sep="\t", na_values="\\N", low_memory=False)
print(f"   Loaded {len(df_basics):,} rows")

print("Loading Ratings Table...")
df_ratings = pd.read_csv(ratings_path, sep="\t", na_values="\\N")
print(f"   Loaded {len(df_ratings):,} rows")

print("\n✅ Data Loaded Successfully")

Loading Basics Table...
   Loaded 12,240,026 rows
Loading Ratings Table...
   Loaded 1,627,920 rows

✅ Data Loaded Successfully


## 3. Initial Inspection

Before cleaning, we must understand what we are dealing with. We check:
1. **Structure:** Columns and rows.
2. **Data Types:** Are numbers actually numbers?
3. **Missing Values:** How many nulls per column?

In [3]:
print("--- Basics Table Info ---")
print(f"Shape: {df_basics.shape[0]:,} rows × {df_basics.shape[1]} columns")
display(df_basics.head(3))
df_basics.info()

--- Basics Table Info ---
Shape: 12,240,026 rows × 9 columns


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892.0,NaN,5,"Animation,Comedy,Romance"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12240026 entries, 0 to 12240025
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         int64  
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(2), int64(1), object(6)
memory usage: 840.5+ MB


In [4]:
print("--- Ratings Table Info ---")
display(df_ratings.head(3))
df_ratings.info()

--- Ratings Table Info ---


,tconst,averageRating,numVotes
0,tt0000001,5.7,2188
1,tt0000002,5.5,308
2,tt0000003,6.5,2288


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627920 entries, 0 to 1627919
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1627920 non-null  object 
 1   averageRating  1627920 non-null  float64
 2   numVotes       1627920 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 37.3+ MB


## 4. Data Cleaning

Now we clean the data step-by-step. 

**Steps:**
1. Filter to movies only (drop TV shows, shorts, video games, etc.)
2. Drop irrelevant columns (`endYear` is for TV series)
3. Merge basics + ratings into one dataframe

### Step 1: Filter to Movies Only

The dataset contains many title types. We only want feature films — our model predicts **movie** ratings, not TV shows or shorts.

In [5]:
# First, let's see what title types exist
print("Title types in dataset:")
print(df_basics['titleType'].value_counts())
print(f"\nTotal rows before filtering: {len(df_basics):,}")

Title types in dataset:
titleType
tvEpisode       9443362
short           1107122
movie            738208
video            321037
tvSeries         293699
tvMovie          153726
tvMiniSeries      67785
tvSpecial         56735
videoGame         47460
tvShort           10891
tvPilot               1
Name: count, dtype: int64

Total rows before filtering: 12,240,026


In [6]:
# Filter to keep only movies
df_movies = df_basics[df_basics['titleType'] == 'movie'].copy()

print(f"✅ Filtered to movies only")
print(f"   Rows: {len(df_basics):,} → {len(df_movies):,}")
print(f"   Reduction: {(1 - len(df_movies)/len(df_basics))*100:.1f}%")

✅ Filtered to movies only
   Rows: 12,240,026 → 738,208
   Reduction: 94.0%


### Step 2: Drop Irrelevant Columns

The `endYear` column indicates when a TV series ended. Movies don't have an end year — they release once. This column is ~95% null for movies and provides no value.

In [7]:
# Drop columns that are irrelevant for movies
# - endYear: only applies to TV series
# - titleType: now all rows are 'movie', so it's redundant

columns_to_drop = ['endYear', 'titleType']
df_movies = df_movies.drop(columns=columns_to_drop)

print(f"✅ Dropped columns: {columns_to_drop}")
print(f"   Remaining columns: {list(df_movies.columns)}")

✅ Dropped columns: ['endYear', 'titleType']
   Remaining columns: ['tconst', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'runtimeMinutes', 'genres']


### Step 3: Merge Basics + Ratings

Our data is split across two tables:
- `df_movies` has **features** (title, year, runtime, genres)
- `df_ratings` has the **target variable** (averageRating, numVotes)

We merge them on `tconst` (the unique movie ID). We use an **inner join** to keep only movies that have ratings.

In [8]:
# Merge movies with their ratings
# Inner join: only keep movies that have ratings data
df = pd.merge(df_movies, df_ratings, on='tconst', how='inner')

print(f"✅ Merged basics + ratings")
print(f"   Movies before merge: {len(df_movies):,}")
print(f"   Movies after merge:  {len(df):,}")
print(f"   Movies without ratings (dropped): {len(df_movies) - len(df):,}")

✅ Merged basics + ratings
   Movies before merge: 738,208
   Movies after merge:  338,468
   Movies without ratings (dropped): 399,740


In [9]:
# Inspect the cleaned, merged dataset
print("--- Cleaned Dataset ---")
display(df.head())
print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
df.info()

--- Cleaned Dataset ---


,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,Miss Jerry,Miss Jerry,0,1894.0,45,Romance,5.2,232
1,tt0000147,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897.0,100,"Documentary,News,Sport",5.3,584
2,tt0000335,Soldiers of the Cross,Soldiers of the Cross,0,1900.0,40,"Biography,Drama",5.4,67
3,tt0000502,Bohemios,Bohemios,0,1905.0,100,NaN,3.1,26
4,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906.0,70,"Action,Adventure,Biography",6.0,1046



Shape: 338,468 rows × 9 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338468 entries, 0 to 338467
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          338468 non-null  object 
 1   primaryTitle    338466 non-null  object 
 2   originalTitle   338466 non-null  object 
 3   isAdult         338468 non-null  int64  
 4   startYear       338433 non-null  float64
 5   runtimeMinutes  305021 non-null  object 
 6   genres          327436 non-null  object 
 7   averageRating   338468 non-null  float64
 8   numVotes        338468 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 23.2+ MB


In [10]:
# Verify: Year distribution of the dataset
print("--- Year Distribution ---")
print(f"Oldest movie: {df['startYear'].min():.0f}")
print(f"Newest movie: {df['startYear'].max():.0f}")
print(f"Median year:  {df['startYear'].median():.0f}")

pct_modern = (df['startYear'] >= 2000).mean() * 100
print(f"\n📊 {pct_modern:.1f}% of movies are from year 2000 or later")

# Show decade distribution
print("\n--- Movies by Decade ---")
decade_counts = (df['startYear'] // 10 * 10).value_counts().sort_index()
print(decade_counts)

--- Year Distribution ---
Oldest movie: 1894
Newest movie: 2026
Median year:  2008

📊 60.6% of movies are from year 2000 or later

--- Movies by Decade ---
startYear
1890.0       18
1900.0      165
1910.0     2112
1920.0     4024
1930.0     9368
1940.0     9245
1950.0    13154
1960.0    17751
1970.0    23584
1980.0    26321
1990.0    27587
2000.0    48124
2010.0    94802
2020.0    62178
Name: count, dtype: int64


## 5. Handling Missing Values

Before dropping missing values, let's understand what we're dealing with.

In [11]:
# Missing values analysis
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=False)

print("--- Missing Values Summary ---")
display(missing_df[missing_df['Missing Count'] > 0])

# Breakdown: both missing vs only one
both_missing = df[df['runtimeMinutes'].isnull() & df['genres'].isnull()]
missing_only_runtime = df[df['runtimeMinutes'].isnull() & df['genres'].notna()]
missing_only_genres = df[df['runtimeMinutes'].notna() & df['genres'].isnull()]

print(f"\n--- Breakdown ---")
print(f"Missing BOTH runtimeMinutes AND genres: {len(both_missing):,}")
print(f"Missing ONLY runtimeMinutes: {len(missing_only_runtime):,}")
print(f"Missing ONLY genres: {len(missing_only_genres):,}")

--- Missing Values Summary ---


,Missing Count,Missing %
runtimeMinutes,33447,9.88
genres,11032,3.26
startYear,35,0.01
primaryTitle,2,0.00
originalTitle,2,0.00



--- Breakdown ---
Missing BOTH runtimeMinutes AND genres: 4,627
Missing ONLY runtimeMinutes: 28,820
Missing ONLY genres: 6,405


In [12]:
# numVotes comparison: movies with missing data vs complete movies
any_missing = df[df['runtimeMinutes'].isnull() | df['genres'].isnull()]
complete = df[df['runtimeMinutes'].notna() & df['genres'].notna()]

print("--- numVotes Comparison ---")
print(f"\nMovies with ANY missing data ({len(any_missing):,} movies):")
print(f"   Median votes: {any_missing['numVotes'].median():.0f}")
print(f"   Mean votes:   {any_missing['numVotes'].mean():.0f}")

print(f"\nComplete movies ({len(complete):,} movies):")
print(f"   Median votes: {complete['numVotes'].median():.0f}")
print(f"   Mean votes:   {complete['numVotes'].mean():.0f}")

# Threshold analysis for movies missing only ONE field
thresholds = [25, 50, 75, 100]
print("\n--- Threshold Analysis ---")
print("For movies missing only ONE field, how many would we keep?")
for thresh in thresholds:
    keep_runtime = (missing_only_runtime['numVotes'] >= thresh).sum()
    keep_genres = (missing_only_genres['numVotes'] >= thresh).sum()
    total = len(missing_only_runtime) + len(missing_only_genres)
    print(f"   ≥{thresh} votes: keep {keep_runtime + keep_genres:,} of {total:,}")

--- numVotes Comparison ---

Movies with ANY missing data (39,852 movies):
   Median votes: 17
   Mean votes:   44

Complete movies (298,616 movies):
   Median votes: 85
   Mean votes:   4183

--- Threshold Analysis ---
For movies missing only ONE field, how many would we keep?
   ≥25 votes: keep 12,717 of 35,225
   ≥50 votes: keep 5,725 of 35,225
   ≥75 votes: keep 3,512 of 35,225
   ≥100 votes: keep 2,499 of 35,225


### Decision: Drop All Rows with Missing runtimeMinutes or genres

**Why we chose a simple drop strategy:**

1. **Abundant data**: With 298k+ complete movies, we have more than enough for any analysis or model
2. **Low-quality records**: Movies with missing data have a median of just 17 votes vs 85 for complete movies — they're overwhelmingly obscure, low-engagement titles
3. **Acceptable loss**: Dropping ~12% of data sounds significant, but we're removing noise, not signal
4. **Simplicity**: A clean pipeline is easier to maintain and explain.

**Alternatives considered:**
- **Hybrid threshold approach**: Keep "missing one field" movies if votes ≥ threshold. Rejected — adds complexity for marginal gain (~3-4% more data that's still incomplete)
- **Imputation**: Fill missing runtime with median, genres with "Unknown". Rejected — introduces assumptions and "Unknown" genre has no predictive value.

In [13]:
# Drop rows with missing runtimeMinutes or genres
rows_before = len(df)
df_clean = df.dropna(subset=['runtimeMinutes', 'genres'])
rows_after = len(df_clean)

print(f"✅ Dropped rows with missing runtimeMinutes or genres")
print(f"   Rows: {rows_before:,} → {rows_after:,}")
print(f"   Dropped: {rows_before - rows_after:,} ({(rows_before - rows_after) / rows_before * 100:.1f}%)")

✅ Dropped rows with missing runtimeMinutes or genres
   Rows: 338,468 → 298,616
   Dropped: 39,852 (11.8%)


In [14]:
# Verify: Check remaining missing values
print("--- Remaining Missing Values ---")
print(df_clean.isnull().sum())
print(f"\n✅ Clean dataset ready: {len(df_clean):,} movies")

--- Remaining Missing Values ---
tconst             0
primaryTitle       0
originalTitle      0
isAdult            0
startYear         22
runtimeMinutes     0
genres             0
averageRating      0
numVotes           0
dtype: int64

✅ Clean dataset ready: 298,616 movies


## 6. Summary

**What we did:**
1. Loaded the full IMDb dataset (12M+ titles, 1.6M+ ratings)
2. Filtered to movies only (738k → 338k after merging with ratings)
3. Dropped irrelevant columns (`endYear`, `titleType`)
4. Analyzed missing data patterns and their relationship to vote counts
5. Dropped ~40k movies with missing `runtimeMinutes` or `genres`

**Final dataset:** ~298k movies with complete data, ready for analysis and modeling.

In [15]:
# Final dataset overview
print("--- Final Clean Dataset ---")
display(df_clean.head())
print(f"\nShape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
df_clean.info()

--- Final Clean Dataset ---


,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,Miss Jerry,Miss Jerry,0,1894.0,45,Romance,5.2,232
1,tt0000147,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897.0,100,"Documentary,News,Sport",5.3,584
2,tt0000335,Soldiers of the Cross,Soldiers of the Cross,0,1900.0,40,"Biography,Drama",5.4,67
4,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906.0,70,"Action,Adventure,Biography",6.0,1046
5,tt0000591,The Prodigal Son,L'enfant prodigue,0,1907.0,90,Drama,4.8,37



Shape: 298,616 rows × 9 columns
<class 'pandas.core.frame.DataFrame'>
Index: 298616 entries, 0 to 338467
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          298616 non-null  object 
 1   primaryTitle    298616 non-null  object 
 2   originalTitle   298616 non-null  object 
 3   isAdult         298616 non-null  int64  
 4   startYear       298594 non-null  float64
 5   runtimeMinutes  298616 non-null  object 
 6   genres          298616 non-null  object 
 7   averageRating   298616 non-null  float64
 8   numVotes        298616 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 22.8+ MB


## 7. Export Clean Dataset

Save the cleaned data so the team can use it directly without re-running the pipeline.

In [16]:
# Export clean dataset to CSV
output_path = "../data/movies_clean.csv"
df_clean.to_csv(output_path, index=False)

print(f"✅ Exported clean dataset to: {output_path}")
print(f"   Rows: {len(df_clean):,}")
print(f"   Size: {round(df_clean.memory_usage(deep=True).sum() / 1024 / 1024, 1)} MB (in memory)")

✅ Exported clean dataset to: ../data/movies_clean.csv
   Rows: 298,616
   Size: 110.2 MB (in memory)
